In [1]:
import numpy as np
import clusters
import datasets as ds
import estimate_k as ek

from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering

In [29]:
# prepare for datasets

# mnist
mnist_train,_,mnist_test,mnist_test_lable = ds.get_mnist()
mnist_train = mnist_train.astype('float32') / 255.
mnist_train = mnist_train.reshape((len(mnist_train),np.prod(mnist_train.shape[1:]))) # make 28*28 img to 784 demission matrix
mnist_test = mnist_test.astype('float32') / 255.
mnist_test = mnist_test.reshape((len(mnist_test), np.prod(mnist_test.shape[1:])))

# usps
usps_train, usps_lable = ds.get_usps()

# cifar20
cifar_train, _, cifar_test, cifar_test_lable = ds.get_cifar20()
cifar_train = cifar_train.astype('float32') / 255.
cifar_train = cifar_train.reshape((len(cifar_train),32*32*3))
cifar_test = cifar_test.astype('float32') / 255.
cifar_test = cifar_test.reshape((len(cifar_test),32*32*3))
cifar_test_lable_1d = []
for lable in cifar_test_lable:
    cifar_test_lable_1d.append(lable[0])

In [3]:
def process(train,test,test_lable,true_k):
    # DE
    de_result = clusters.de(train,test) # train result is based on 'test', so clustering score should base on 'test' too
    print(de_result.shape) # de_result is embeded test

    # --- estimate k ---

    # DE+SA
    k = ek.get_k_from_result(ek.estimate(2*true_k, de_result))
    print("DE+SA, estimate k :")
    print(k)

    # SA
    k = ek.get_k_from_result(ek.estimate(2*true_k,test))
    print("SA, estimate k :")
    print(k)

    # TODO:X-means

    # TODO:G-means

    # --- clustering ---

    # K-means
    km = KMeans(n_clusters=true_k,init='k-means++', max_iter=200)
    km_result = km.fit_predict(test)
    score = clusters.score(test_lable,km_result)
    print("k-means score with true k:")
    print(score)

    # DE, K-means
    km = KMeans(n_clusters=true_k,init='k-means++', max_iter=200)
    de_km_result = km.fit_predict(test)
    score = clusters.score(test_lable,de_km_result)
    print("DE k-means score with true k:")
    print(score)

    # SCDE clustering
    sc = SpectralClustering(n_clusters=true_k, eigen_solver='arpack', affinity="nearest_neighbors")
    scde_result = sc.fit_predict(de_result)
    score = clusters.score(test_lable,scde_result)
    print("scde score with true k:")
    print(score)

    # SC clustering
    sc = SpectralClustering(n_clusters=true_k, eigen_solver='arpack', affinity="nearest_neighbors")
    sc_result = sc.fit_predict(test)
    score = clusters.score(test_lable,sc_result)
    print("sc score with true k:")
    print(score)

    return

In [5]:
# process on mnist
process(mnist_train,mnist_test,mnist_test_lable,10)

Epoch 1/50
235/235 [==============================] - 16s 67ms/step - loss: 52.3480
Epoch 2/50
235/235 [==============================] - 21s 88ms/step - loss: 47.8340
Epoch 3/50
235/235 [==============================] - 20s 83ms/step - loss: 44.2055
Epoch 4/50
235/235 [==============================] - 19s 80ms/step - loss: 40.9872
Epoch 5/50
235/235 [==============================] - 21s 87ms/step - loss: 37.1539
Epoch 6/50
235/235 [==============================] - 19s 80ms/step - loss: 33.5707
Epoch 7/50
235/235 [==============================] - 19s 81ms/step - loss: 31.2491
Epoch 8/50
235/235 [==============================] - 19s 80ms/step - loss: 29.7918
Epoch 9/50
235/235 [==============================] - 19s 81ms/step - loss: 28.4802
Epoch 10/50
235/235 [==============================] - 19s 80ms/step - loss: 27.3504
Epoch 11/50
235/235 [==============================] - 19s 81ms/step - loss: 26.4007
Epoch 12/50
235/235 [==============================] - 19s 80ms/step - los

In [4]:
# process on usps
process(usps_train, usps_train, usps_lable, 10)

Epoch 1/50
29/29 [==============================] - 1s 48ms/step - loss: 34.7282
Epoch 2/50
29/29 [==============================] - 1s 47ms/step - loss: 29.2039
Epoch 3/50
29/29 [==============================] - 1s 45ms/step - loss: 28.6469
Epoch 4/50
29/29 [==============================] - 1s 44ms/step - loss: 28.0623
Epoch 5/50
29/29 [==============================] - 1s 47ms/step - loss: 28.1090
Epoch 6/50
29/29 [==============================] - 1s 45ms/step - loss: 27.1539
Epoch 7/50
29/29 [==============================] - 1s 43ms/step - loss: 26.2070
Epoch 8/50
29/29 [==============================] - 1s 45ms/step - loss: 25.6904
Epoch 9/50
29/29 [==============================] - 1s 45ms/step - loss: 25.2903
Epoch 10/50
29/29 [==============================] - 1s 46ms/step - loss: 24.9228
Epoch 11/50
29/29 [==============================] - 1s 45ms/step - loss: 24.5617
Epoch 12/50
29/29 [==============================] - 2s 54ms/step - loss: 24.2637
Epoch 13/50
29/29 [======

In [30]:
# process on cifar20
process(cifar_train, cifar_test, cifar_test_lable_1d, 40)

Epoch 1/50
196/196 [==============================] - 30s 154ms/step - loss: nan
Epoch 2/50
196/196 [==============================] - 38s 193ms/step - loss: nan
Epoch 3/50
196/196 [==============================] - 39s 197ms/step - loss: nan
Epoch 4/50
196/196 [==============================] - 38s 192ms/step - loss: nan
Epoch 5/50
196/196 [==============================] - 34s 174ms/step - loss: nan
Epoch 6/50
196/196 [==============================] - 39s 197ms/step - loss: nan
Epoch 7/50
196/196 [==============================] - 40s 203ms/step - loss: nan
Epoch 8/50
196/196 [==============================] - 37s 191ms/step - loss: nan
Epoch 9/50
196/196 [==============================] - 37s 188ms/step - loss: nan
Epoch 10/50
196/196 [==============================] - 38s 193ms/step - loss: nan
Epoch 11/50
196/196 [==============================] - 37s 191ms/step - loss: nan
Epoch 12/50
196/196 [==============================] - 40s 202ms/step - loss: nan
Epoch 13/50
196/196 [====

ResourceExhaustedError: OOM when allocating tensor with shape[256,3072] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:BiasAdd]